In [ ]:
#!git clone https://github.com/AlexanderZah/InternVL-interp.git

In [ ]:
import os
os.environ['ROOT_DIR'] = '/content/InternVL-interp'

In [ ]:
#!pip install -r $ROOT_DIR/requirements.txt

In [ ]:
import sys

sys.path.append(os.environ["ROOT_DIR"])
sys.path.append(os.environ["ROOT_DIR"] + '/methods')
sys.path.append(os.environ["ROOT_DIR"] + 'src/caption/internvl')

: 

In [ ]:
import os

from methods.internvl_utils import retrieve_logit_lens_internvl, load_internvl_state
from methods.algorithms import internal_confidence, internal_confidence_heatmap, internal_confidence_segmentation
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

: 

In [ ]:
model_state = load_internvl_state()

/kaggle/working/vl-interp


config.json:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-4B:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-4B:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/9.02k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.38M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

In [ ]:
# %cd /kaggle/working/vl-interp
# !git pull


/kaggle/working/vl-interp
Already up to date.


In [ ]:
# import sys
# import importlib
# for module in [m for m in sys.modules.keys() if m.startswith('methods') or m.startswith('internvl') or m.startswith('qwenvl')]:
    
#     if module in sys.modules:
#         print(module)
#         importlib.reload(sys.modules[module])

methods
methods.utils
methods.internvl_utils
methods.algorithms
methods.qwen_vl_utils


In [ ]:
img_path = "images/COCO_val2014_000000004108.jpg"
caption, softmax_probs = retrieve_logit_lens_internvl(model_state, img_path, num_patches=1)

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:463: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
`generation_config` default values have been modified to match model-specific defaults: {'bos_token_id': 151643}. If this is not desired, please set these values explicitly.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Caption: The image depicts a festive gathering with a table filled with various food items and drinks. In the center of the table, there is a large chocolate cake with green icing, which appears to be the centerpiece of the event. Surrounding the cake are several plates with slices of cake, as well as plates with what looks like a dessert or sweet treat. 

To the right of the cake, there is a large tray filled with hot dogs, which are arranged neatly on a large green leaf. The hot dogs are accompanied by condiments, including a bottle of ketchup and a jar of mustard. There are also several cups with straws, suggesting that beverages are being served. 

In the background, there are people seated at tables, enjoying the event. The room is decorated with balloons, indicating that this is a celebratory occasion, possibly a birthday party or a similar event. The overall atmosphere is lively and convivial, with people engaged in conversation and enjoying the food and company.


In [ ]:
class_ = ' food'
internal_confidence(model_state['tokenizer'], softmax_probs, class_)

In [ ]:

model_name = model_state['model_name']
heatmap_data = internal_confidence_heatmap(model_state['tokenizer'], softmax_probs, class_)
num_image_embeddings = softmax_probs.shape[2]
im = plt.imshow(heatmap_data, aspect=30/num_image_embeddings, cmap='Blues', interpolation='nearest')
plt.title(f"'{class_}' probabilities")
plt.xlabel("LM Layer")
plt.ylabel("Image Embedding Index")
plt.tight_layout()
plt.clim(0, 1)
plt.colorbar()
plt.savefig(f'/kaggle/working/{model_name}/heatmap_{class_}.png')
plt.show()

In [ ]:
img = Image.open(img_path).convert("RGB")
img_width, img_height = img.size

segmentation = internal_confidence_segmentation(model_state['tokenizer'], softmax_probs, class_, 16)
segmentation_resized = (np.array(Image.fromarray(segmentation).resize((img_width, img_height), Image.BILINEAR)))

plt.imshow(img)
plt.imshow(segmentation_resized, cmap='jet', interpolation='bilinear', alpha=.5)
plt.axis('off')
plt.title(f"'{class_}' localization")
plt.tight_layout()
plt.savefig(f'/kaggle/working/{model_name}/segmentation_{class_}.png')
plt.show()